In [10]:
from datetime import datetime, date

In [1]:
actions = pd.read_csv('/Users/dima/Downloads/actions.csv')
actions.shape


(2200, 35)

In [2]:
actions.columns

Index(['Event Name', 'Time', 'Distinct ID', 'Browser', 'Browser Version',
       'City', 'Current URL', 'Device ID', 'Initial Referrer',
       'Initial Referring Domain', 'Insert ID', 'Library Version',
       'API Endpoint', 'API Timestamp', 'Operating System', 'Referrer',
       'Referring Domain', 'Region', 'Screen Height', 'Screen Width',
       'User ID', 'account', 'cta', 'id', 'module', 'Country',
       'Mixpanel Library', 'Time Processed (UTC)', 'Sent By Library Version',
       'phase', 'preset', 'req_id', 'screen', 'Had Persisted Distinct Id',
       'Search Engine'],
      dtype='object')

In [8]:
actions = actions[actions['account'] == 'dallas']
actions.shape

(248, 35)

In [9]:
mongo = gmcdb('production', 'dallas')

2024-02-11T13:20:51.052820Z [info     ] Mongos instances selected      environment=research_sb1 file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/connection_string_builder.py function_name=get_selected_mongos_instances hostname=Dima-Shulga-MacBook-Pro hosts=['applicativedb-prod-mongos-4.omcomcom.com', 'applicativedb-prod-mongos-2.omcomcom.com'] line_number=38 module=hs_gimme.db_facade.connection_string_builder pid=79241


In [12]:
req_fields =  ['job_department',
              'country',
               'external_job_band',
               'external_recruiters',
               'top_category',
               'sub_category',
               'job_education',
               'seniority_level',
               'min_years_of_relevant_experience',
               'max_years_of_relevant_experience',
               'max_salary',
               'job_type',
               'is_visa_required',
               'is_remote_location', 
               'external_status',
               'industry',
               'open_date',
               'past_candidates_distribution_date',
               'is_recent_grad', 
               'is_intern',
               '_created_at',
               'date_posted',
               'job_create_date',
               'last_edit_date',
               'status_history',
               'recruiter_roles',
               'recruiting_type']

In [11]:
reference_date = datetime(2023, 11, 1)

In [20]:
reqs = list(mongo.req.find({'_created_at': {'$gt': reference_date}, 'date_posted': 
                            {'$gt': reference_date}, 'external_status': 'Filled'
                           }, req_fields))
len(reqs)

56

In [21]:
req_ids = [r['_id'] for r in reqs]

In [22]:
app_fields = ['req_id', 'status_history', 'is_internal']

In [23]:
apps = list(mongo.application.find({'req_id': {'$in': req_ids}}, app_fields))
len(apps)

4049

In [24]:
reqs[0]['status_history']

{'Open': [datetime.datetime(2023, 10, 31, 0, 0)],
 'Approve': [datetime.datetime(2023, 10, 31, 0, 0)],
 'Post': [datetime.datetime(2023, 11, 27, 0, 0),
  datetime.datetime(2023, 11, 29, 0, 0)],
 'Filled': [datetime.datetime(2024, 2, 4, 8, 3, 33, 875000)]}

In [28]:
(datetime(2024, 2, 4, 8, 3, 33, 875000) - datetime(2023, 11, 27, 0, 0)).days

69

In [37]:
for r in reqs: 
    r['time_to_fill'] = (r['status_history']['Filled'][0] - r['status_history']['Post'][0]).days
    r['num_of_actions'] = len(actions[actions['req_id'] == r['_id']])

In [38]:
reqs_df = pd.DataFrame(reqs)

In [48]:
actions[['id']].iloc[4].values

array(['pending_candidates_notification__review_candidate_internal_PendingNotificationScopeType.SEPERATE_APPLICATIONS__0'],
      dtype=object)

In [40]:
reqs_df['num_of_actions'].describe()

count    56.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: num_of_actions, dtype: float64

In [49]:
reqs_df['user'] = reqs_df['recruiter_roles'].apply(lambda x: x['recruiter'])

In [54]:
reqs_df.groupby('user')[['time_to_fill', 'num_of_actions']].mean()

,time_to_fill,num_of_actions
user,,
0823038bdcfd01cb5bd686826e0983bb,66.000000,0.0
08bfb4dd3be701011b20f1429e600000,49.000000,0.0
13b6054a2ff1016ef676bc427f29b155,75.000000,0.0
16e250631a9e011b63050bd1c2076491,40.000000,0.0
17ff7f4beac501627e69765409017033,51.000000,0.0
21065fc29a6201f17c9c5dc9121510ca,45.000000,0.0
395e9d683c150183192170e95208352d,35.000000,0.0
4e9f9f2c74050148c90cd2b5c605ab2d,28.000000,0.0
7491de069071014dd2c05d9bd516054e,48.500000,0.0


In [55]:
actions

,Event Name,Time,Distinct ID,Browser,Browser Version,City,Current URL,Device ID,Initial Referrer,Initial Referring Domain,...,Country,Mixpanel Library,Time Processed (UTC),Sent By Library Version,phase,preset,req_id,screen,Had Persisted Distinct Id,Search Engine
49,my-notifications.click,1.707400e+09,DEL546,Microsoft Edge,121.0,Mumbai,https://cloud.hiredscore.com/dell/#/pendingAct...,18a213a0b5f793-0463a33054b7a3-7c54647e-e1000-1...,https://pf.us.dell.com/,pf.us.dell.com,...,IN,web,1707400228200,2.49.0,NaN,homescreen,NaN,pending_actions,NaN,NaN
57,my-notifications.click,1.707345e+09,DEL25,Chrome,121.0,Franklin,https://cloud.hiredscore.com/dell/#/req/DELR23...,18b913ced4a667-0a3566062d6f82-26031151-172698-...,https://pf.us.dell.com/,pf.us.dell.com,...,US,web,1707345485382,2.49.0,NaN,homescreen,NaN,req_inbox,NaN,NaN
70,my-notifications.click,1.707312e+09,DEL299,Chrome,121.0,Jihlava,https://cloud.hiredscore.com/dell/#/req/DELR23...,18a604db9a98bf-00af49f7690ed9-26031f51-e1000-1...,https://pf.us.dell.com/,pf.us.dell.com,...,CZ,web,1707311869722,2.49.0,NaN,homescreen,NaN,req_inbox,NaN,NaN
81,my-notifications.click,1.707251e+09,DEL25,Chrome,121.0,Franklin,https://cloud.hiredscore.com/dell/#/pendingAct...,18b913ced4a667-0a3566062d6f82-26031151-172698-...,https://pf.us.dell.com/,pf.us.dell.com,...,US,web,1707250827228,2.49.0,NaN,homescreen,NaN,pending_actions,NaN,NaN
120,my-notifications.click,1.707156e+09,DEL676,Chrome,121.0,New York,https://cloud.hiredscore.com/dell/#/pendingAct...,18a51eaef295a6-0b330bfec973a5-26031f51-1fa400-...,https://pf.us.dell.com/,pf.us.dell.com,...,US,web,1707156291956,2.48.1,NaN,homescreen,NaN,pending_actions,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,my-notifications.click,1.698759e+09,DEL73,Chrome,118.0,New York,https://cloud.hiredscore.com/dell/#/pendingAct...,18a70cdca334b3-02dae62c587bf1-26031f51-1fa400-...,https://pf.us.dell.com/,pf.us.dell.com,...,US,web,1698758609011,2.47.0,NaN,homescreen,NaN,pending_actions,NaN,NaN
2171,my-notifications.click,1.698732e+09,DEL84,Chrome,118.0,Bengaluru,https://cloud.hiredscore.com/dell/#/pendingAct...,18ab10c14f37ca-0ffcb215199624-26031f51-e1000-1...,https://pf.us.dell.com/,pf.us.dell.com,...,IN,web,1698731542426,2.47.0,NaN,homescreen,NaN,pending_actions,NaN,NaN
2185,my-notifications.click,1.698699e+09,DEL8,Microsoft Edge,118.0,Portland,https://cloud.hiredscore.com/dell/#/pendingAct...,18b825d2d943b1-0581bdccd29cbe-745d5777-144000-...,https://pf.us.dell.com/,pf.us.dell.com,...,US,web,1698699237755,2.47.0,NaN,homescreen,NaN,pending_actions,NaN,NaN
2195,my-notifications.click,1.698688e+09,DEL10,Chrome,118.0,Edmond,https://cloud.hiredscore.com/dell/#/pendingAct...,18a850d05107ed-03149e97585787-26031f51-144000-...,https://pf.us.dell.com/,pf.us.dell.com,...,US,web,1698688409661,2.47.0,NaN,homescreen,NaN,pending_actions,NaN,NaN
